# OpenRouteService Optimization Routing examples

This notebook is just a small reference on how to use `openrouteservice` package to request:
- [directions](https://openrouteservice.org/dev)
- [Route Optimization](https://openrouteservice.org/dev/#/api-docs/optimization/post)

In [1]:
import openrouteservice as ors
import folium

povoLoc = [46.064424,11.157259]

center = povoLoc

client = ors.Client(base_url='http://localhost:8080/ors', key='')

/home/z4k/.local/lib/python2.7/site-packages/folium/__init__.py:59: UserWarning: This version of folium is the last to support Python 2. Transition to Python 3 to be able to receive updates and fixes. Check out https://python3statement.org/ for more info.
  UserWarning


## TOC

- **[Directions](#Direction)**
- **[Optimization](#Optimization)**

## Directions

In [3]:
m = folium.Map(location=center, tiles='OpenStreetMap', zoom_start=16)

# Some coordinates in Berlin
coordinates = [
    [11.157968044281006, 46.064223853791624],
    [11.151208877563475, 46.06742477567994]
]

route = client.directions(
    coordinates=coordinates,
    profile='driving-car',
    format='geojson',
    #options={"avoid_features": ["steps"]},
    validate=False,
)
folium.PolyLine(locations=[list(reversed(coord)) 
                           for coord in 
                           route['features'][0]['geometry']['coordinates']]).add_to(m)
m

## Optimization

### Simple waypoint optimization

With `openrouteservice-py` you can ask for waypoint optimization when requesting a normal `direction`. This assumes that your first coordinate is the start location and the last coordinate is the end location, i.e. only the `via` endpoints are optimized. To make it a round trip, make the first and last location the same.

In [23]:
m = folium.Map(location=center, tiles='OpenStreetMap', zoom_start=16)

# Some coordinates in Berlin
coordinates = [[13.384116, 52.533558], [13.41774, 52.498929], [13.428726, 52.519355], [13.374825, 52.496369]]

# The popup will show the ID in the coordinate list. In a non-optimized waypoint order, the waypoints
# would have been visited from ID 0 to ID 3 sequentially.
for idx, coords in enumerate(coordinates):
    folium.Marker(location=list(reversed(coords)),
                 popup=folium.Popup("ID: {}".format(idx))).add_to(m)

route = client.directions(
    coordinates=coordinates,
    profile='driving-car',
    format='geojson',
    validate=False,
    optimize_waypoints=True
)

folium.PolyLine(locations=[list(reversed(coord)) 
                           for coord in 
                           route['features'][0]['geometry']['coordinates']]).add_to(m)
    
m

### Optimize job scheduling for multiple vehicles

The next example makes more use of the power of the ORS optimization endpoint. Two vehicles are assigned to 6 jobs, where the capacity constraints are such that each vehicle can only carry out 3 jobs on its route.

In [24]:
m = folium.Map(location=center, tiles='OpenStreetMap', zoom_start=16)

vehicle_locations = [[13.390446, 52.506087], [13.384116, 52.533558]]
job_locations = [[13.428726, 52.519355],
                 [13.41774, 52.498929],
                 [13.374825, 52.496369],
                 [13.378859, 52.509796],
                 [13.400488, 52.509691],
                 [13.358517, 52.524264]]

# Assign vehicles to do the jobs
vehicles = []
for idx, coords in enumerate(vehicle_locations):
    vehicles.append(ors.optimization.Vehicle(
        id=idx,
        profile='driving-car',
        start=coords,
        end=coords,
        capacity=[3]  # Limit capacity so only 3 jobs can be taken by each vehicle
    ))
    folium.Marker(location=list(reversed(coords)), icon=folium.Icon(icon='truck', prefix='fa')).add_to(m)

# Define jobs to be carried out
jobs=[]
for idx, coords in enumerate(job_locations):
    jobs.append(ors.optimization.Job(
        id=idx,
        location=coords,
        amount=[1]  # Occupies capacity in vehicle
    ))
    folium.Marker(location=list(reversed(coords)), icon=folium.Icon(icon='archive', prefix='fa', color='green')).add_to(m)

optimized = client.optimization(
    jobs=jobs,
    vehicles=vehicles,
    geometry=True,  ## will output the geometry,
)

folium.PolyLine(
    locations=[list(reversed(coords)) for coords in ors.convert.decode_polyline(optimized['routes'][0]['geometry'])['coordinates']],
    color='red'
).add_to(m)

folium.PolyLine(
    locations=[list(reversed(coords)) for coords in ors.convert.decode_polyline(optimized['routes'][1]['geometry'])['coordinates']],
    color='orange'
).add_to(m)

m